# Chat with BanterBot8000

Run the first cell to initialize BanterBot8000, and then have fun chatting below!

In [7]:
from src import init_llm, banter
from guidance import assistant, user
from time import sleep

llm = init_llm()

<div style="display: flex; align-items: top;">
  <img src="banterbot8000.png" alt="BanterBot8000" width="80" style="margin-right: 20px;">
  <p>Chat with BanterBot8000! He's quite the funny fellow! Run the below code to begin. Say "bye" at any time to quit. </p>
</div>

In [2]:
with assistant():
    banterbot8000 = llm + 'Hi there! Have a joke you want me to explain? Or just here for a laugh?'

while True:
    user_input = input('Talk to Banterbot8000: ')
        
    if user_input.lower() in ['exit', 'quit', '', 'bye']:
        break
    
    with user():
        banterbot8000 += user_input
    
    banterbot8000 += banter(user_input)
    
    sleep(1)

KeyboardInterrupt: 

In [15]:
from src.homophones import get_homophones

get_homophones('two tired')

['too tired',
 'dew tired',
 'do tired',
 'due tired',
 'chew tired',
 'coo tired',
 'coup tired',
 'leu tired',
 'lieu tired',
 'loo tired',
 'pu tired',
 'rue tired',
 'sue tired',
 'toot iron']

In [ ]:
from guidance import  select, gen, system

explanation = '''
    Puns are jokes that play on the multiple meanings of words or the sounds of words that sound alike.
    Which type of pun is "guts"? Is it humorous because it sounds like another word, or because it has multiple meanings?

    Examples:
    Q: What do you call a fake noodle? An impasta.
    A: The pun "impasta" sounds like "imposter."
      
    Q: I used to be a baker, but I couldn't make enough dough.
    A: The pun "dough" has two meanings "money" and "baking ingredient."
'''

with system():
    lm = llm + explanation

with user():
    lm += 'Why didn\'t the skeletons go skydiving? They didn\'t have the guts'
    
with assistant():
    lm += 'A: The pun "guts" ' + select(["m", "s"])

with user():
    lm += 'makes sense'